In [8]:
import pandas as pd
import pulp
from pulp import GLPK
import numpy as np

In [9]:
df = pd.read_csv("Dea_data.csv")
df = df.drop(columns=['Unnamed: 0'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 13 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Company                                         25 non-null     object 
 1   Year                                            25 non-null     int64  
 2   GHG_Intensity                                   25 non-null     float64
 3   Energy_Intensity_per_Sales                      25 non-null     float64
 4   NonRenewable_Energy_Consumption_and_Production  25 non-null     float64
 5   Water_Intensity_Per_Sales                       25 non-null     float64
 6   NonRecycled_Waste_ration                        25 non-null     float64
 7   ESG_Score                                       25 non-null     float64
 8   GHG_1                                           25 non-null     float64
 9   GHG_2                                        

In [10]:
df.replace(0, 0.01, inplace=True)
df.head()

,Company,Year,GHG_Intensity,Energy_Intensity_per_Sales,NonRenewable_Energy_Consumption_and_Production,Water_Intensity_Per_Sales,NonRecycled_Waste_ration,ESG_Score,GHG_1,GHG_2,GHG_3,1_plus_2,1_plus_2_plus_3
0,SONY US Equity,2023,283.35,96.22,89.9,0.010,3.3,5.09,2.72,8.73,250.54,11.45,261.99
1,SONY US Equity,2022,255.60,95.00,95.1,3.392,2.0,5.03,2.63,12.28,227.68,14.92,242.60
2,SONY US Equity,2021,283.36,40.75,94.4,6.784,2.0,4.97,2.36,10.44,226.74,12.80,239.54
3,SONY US Equity,2020,237.57,40.61,95.8,10.176,2.1,4.79,2.41,15.22,190.47,17.63,208.09
4,SONY US Equity,2019,263.10,40.76,98.1,13.568,4.6,4.51,2.48,15.58,216.95,18.05,235.00


In [11]:
df.shape

(25, 13)

In [12]:
class DEA_models:
    def __init__(self):
        DEFALUT_inputs_t1 = np.array(df[df['Year'] == 2019][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                                'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
        DEFALUT_outputs_t1 = np.array(df[df['Year'] == 2019][['GHG_Intensity', 'ESG_Score']])
        self.num_dmus = DEFALUT_inputs_t1.shape[0]
        self.num_inputs = DEFALUT_inputs_t1.shape[1]
        self.num_outputs = DEFALUT_outputs_t1.shape[1]

        self.result_matrix = pd.DataFrame(columns=['SONY', 'APPL', 'DELL', 'HPE', 'SAMSUNG', 'EorT'])

    def calculate_efficiency(self, inputs, outputs, reference_inputs, reference_outputs,i):
        model = pulp.LpProblem("Efficiency_Calculation", pulp.LpMaximize)
        lambdas = [pulp.LpVariable(f"lambda_{j}", lowBound=0) for j in range(self.num_dmus)]
        model += pulp.lpSum([lambdas[j] * reference_outputs[j, 0] for j in range(self.num_dmus)])
        for k in range(self.num_inputs):
            model += pulp.lpSum([lambdas[j] * reference_inputs[j, k] for j in range(self.num_dmus)]) <= inputs[i, k]
        model += pulp.lpSum(lambdas) == 0.1
        model.solve()
            
        if model.status == pulp.LpStatusOptimal:
            self.efficiency_score = pulp.value(pulp.lpSum([lambdas[j] * reference_outputs[j, 0] for j in range(self.num_dmus)])) / outputs[i, 0]
        else:
            self.efficiency_score = -1
            
        return self.efficiency_score
    
    def return_periods(self, year_):
        inputs_t1 = np.array(df[df['Year'] == year_][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                            'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
        outputs_t1 = np.array(df[df['Year'] == year_][['GHG_Intensity', 'ESG_Score']])
        inputs_t2 = np.array(df[df['Year'] == (year_+1)][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                            'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
        outputs_t2 = np.array(df[df['Year'] == (year_+1)][['GHG_Intensity', 'ESG_Score']])

        return inputs_t1, outputs_t1, inputs_t2, outputs_t2
    
    def Malquist_result(self):
        collection_slices = []
        for i in [2019, 2020, 2021, 2022]:
            in_t1, out_t1, in_t2, out_t2 = self.return_periods(i)

            malmquist_indices = []
            EC_list = []
            TC_list = []
            for i in range(self.num_dmus):
                eff_t1_t1 = self.calculate_efficiency(in_t1, out_t1, in_t1, out_t1, i)
                eff_t1_t2 = self.calculate_efficiency(in_t1, out_t1, in_t2, out_t2, i)
                eff_t2_t1 = self.calculate_efficiency(in_t2, out_t2, in_t1, out_t1, i)
                eff_t2_t2 = self.calculate_efficiency(in_t2, out_t2, in_t2, out_t2, i)
                #print(eff_t1_t1, eff_t1_t2, eff_t2_t1, eff_t2_t2)
                if all(v is not None for v in [eff_t1_t1, eff_t2_t2, eff_t1_t2, eff_t2_t1]):
                    EC = eff_t2_t2 / eff_t1_t1 if eff_t1_t1 != 0 else None
                    TC = np.sqrt((eff_t1_t2 * eff_t2_t2) / (eff_t1_t1 * eff_t2_t1)) if all(v != 0 for v in [eff_t1_t1, eff_t2_t1]) else None
                    #print(EC, TC)
                    EC_list.append(EC)
                    TC_list.append(TC)
                    malmquist_index = EC * TC if EC is not None and TC is not None else None
                    malmquist_indices.append(malmquist_index)
                else:
                    malmquist_indices.append(None)
            malmquist_indices.append('ETC')
            collection_slices.append(malmquist_indices)
            EC_list.append('EC')
            TC_list.append('TC')
            self.result_matrix.loc[len(self.result_matrix)] = malmquist_indices
            self.result_matrix.loc[len(self.result_matrix)] = EC_list
            self.result_matrix.loc[len(self.result_matrix)] = TC_list
    

In [13]:
Malquits_model = DEA_models()
Malquits_model.Malquist_result()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/15052a05f8ea444493b5e4f30441195f-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/15052a05f8ea444493b5e4f30441195f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 41 RHS
At line 47 BOUNDS
At line 48 ENDATA
Problem MODEL has 5 rows, 5 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 5 (0) columns and 25 (0) elements
0  Obj -0 Primal inf 0.096715715 (1) Dual inf 3515.256 (5)
0  Obj -0 Primal inf 0.096715715 (1) Dual inf 1.3835202e+11 (5)
1  Obj 32.783
Optimal - objective value 32.783
Optimal objective 32.783 - 1 iterations time 0.002
Option for printingOptions c

/var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/ipykernel_31206/1664545764.py:54: RuntimeWarning: invalid value encountered in sqrt
  TC = np.sqrt((eff_t1_t2 * eff_t2_t2) / (eff_t1_t1 * eff_t2_t1)) if all(v != 0 for v in [eff_t1_t1, eff_t2_t1]) else None


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/fd2062baf45b4912854ccb7cfee0ec24-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/fd2062baf45b4912854ccb7cfee0ec24-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 41 RHS
At line 47 BOUNDS
At line 48 ENDATA
Problem MODEL has 5 rows, 5 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 5 (0) columns and 25 (0) elements
0  Obj -0 Primal inf 0.08761271 (1) Dual inf 9383.769 (5)
0  Obj -0 Primal inf 0.08761271 (1) Dual inf 2.7841655e+11 (5)
4  Obj 50.485706
Optimal - objective value 50.485706
Optimal objective 50.48570602 - 4 iterations time 0.002
Option for printing

In [14]:
Malquits_model.result_matrix

,SONY,APPL,DELL,HPE,SAMSUNG,EorT
0,1.621647,1.599095,0.921900,1.210078,1.503502,ETC
1,1.340117,1.321480,0.761852,1.000000,1.242483,EC
2,1.210078,1.210078,1.210078,1.210078,1.210078,TC
3,0.933931,1.513708,1.337081,1.649274,0.725238,ETC
4,0.880896,1.354675,1.147991,1.249138,0.422662,EC
5,1.060206,1.117396,1.164714,1.320330,1.715881,TC
6,0.849410,1.010342,0.696094,0.898876,0.921976,ETC
7,0.926310,1.148458,0.789184,1.009611,1.000000,EC
8,0.916983,0.879738,0.882044,0.890319,0.921976,TC
9,NaN,1.989968,1.343128,1.046648,1.155757,ETC
